In [5]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [6]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [7]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [8]:
#Dataloader

#Path for training and testing directory
train_path=r"D:\OUTPUT\train"
test_path=r"D:\OUTPUT\test"
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [9]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [10]:
print(classes)

['Maize', 'Okra', 'Peanut']


In [11]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=3):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            

In [12]:
model=ConvNet(num_classes=3).to(device)

In [13]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [14]:
num_epochs=10

In [15]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [16]:
print(train_count,test_count)

234 78


In [17]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),r'D:\best_checkpoint.model')
        best_accuracy=test_accuracy
        

Epoch: 0 Train Loss: tensor(18.1535) Train Accuracy: 0.36324786324786323 Test Accuracy: 0.3333333333333333
Epoch: 1 Train Loss: tensor(3.7730) Train Accuracy: 0.7264957264957265 Test Accuracy: 0.3333333333333333
Epoch: 2 Train Loss: tensor(3.5965) Train Accuracy: 0.688034188034188 Test Accuracy: 0.3333333333333333
Epoch: 3 Train Loss: tensor(2.7954) Train Accuracy: 0.7948717948717948 Test Accuracy: 0.5
Epoch: 4 Train Loss: tensor(1.6701) Train Accuracy: 0.8461538461538461 Test Accuracy: 0.717948717948718
Epoch: 5 Train Loss: tensor(0.8503) Train Accuracy: 0.8974358974358975 Test Accuracy: 0.5641025641025641
Epoch: 6 Train Loss: tensor(0.3941) Train Accuracy: 0.9188034188034188 Test Accuracy: 0.6025641025641025
Epoch: 7 Train Loss: tensor(0.4152) Train Accuracy: 0.9487179487179487 Test Accuracy: 0.5512820512820513
Epoch: 8 Train Loss: tensor(0.2238) Train Accuracy: 0.9572649572649573 Test Accuracy: 0.5897435897435898
Epoch: 9 Train Loss: tensor(0.0776) Train Accuracy: 0.9871794871794872

NameError: name 'test_accuracy' is not defined

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [46]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [47]:
train_path=r"D:\OUTPUT\train"
pred_path=r"D:\CROP CLASSIFICATION1.v1i.folder\seg_pred"

In [48]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [49]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=3):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [50]:
checkpoint=torch.load(r"D:\best_checkpoint.model")
model=ConvNet(num_classes=3)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=3, bias=True)
)

In [51]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [52]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [53]:
images_path=glob.glob(pred_path+'/*.jpg')

In [54]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [55]:
pred_dict


{'D:\\CROP CLASSIFICATION1.v1i.folder\\seg_pred\\agument_original_final maize14.jpg_372135f4-4e31-40bc-ac0b-c8f1657e31b5.jpg': 'Maize',
 'D:\\CROP CLASSIFICATION1.v1i.folder\\seg_pred\\agument_original_final maize15.jpg_26f1aa3f-5978-4712-888e-6d88ae08fb4d.jpg': 'Maize',
 'D:\\CROP CLASSIFICATION1.v1i.folder\\seg_pred\\agument_original_final maize16.jpg_e65df747-ff35-4fc9-8e2e-20045f357c0a.jpg': 'Maize',
 'D:\\CROP CLASSIFICATION1.v1i.folder\\seg_pred\\agument_original_final maize19.jpg_3be9e29f-f04b-4702-88fb-913eed45cd41.jpg': 'Maize',
 'D:\\CROP CLASSIFICATION1.v1i.folder\\seg_pred\\agument_original_final maize23.jpg_00809f26-be39-4f5a-99ff-05f777fb0a23.jpg': 'Maize',
 'D:\\CROP CLASSIFICATION1.v1i.folder\\seg_pred\\agument_original_final maize24.jpg_dff281fa-ea5e-496d-bf91-667dc89658a1.jpg': 'Maize',
 'D:\\CROP CLASSIFICATION1.v1i.folder\\seg_pred\\agument_original_final maize29.jpg_26109857-01a8-462e-be7e-2b1b41232a7c.jpg': 'Maize',
 'D:\\CROP CLASSIFICATION1.v1i.folder\\seg_pred\

# word to excel

In [58]:
import pandas as pd
df = pd.read_csv(r"D:\P1.txt") # can replace with df = pd.read_table('input.txt') for '\t'
# print(df)
print(df.to_excel(r"D:\EXCEL VALUES IMAGES\output2.xlsx"))

None


# probability predictions 

In [53]:
import pandas as pd
confusion_matrix=pd.crosstab(pred_dict["actual_class"],pred_dict["predicted_class"],rownames=["Actual"],colnames=["Predicted"])

KeyError: 'actual_class'

In [5]:
acc = history.history[‘acc’]
val_acc = history.history[‘val_acc’]
loss = history.history[‘loss’]
val_loss = history.history[‘val_loss’]
epochs = range(len(acc))
plt.plot(epochs, acc, ‘r’, label=’Training acc’)
plt.plot(epochs, val_acc, ‘b’, label=’Validation acc’)
plt.title(‘Training and validation accuracy’)
plt.ylabel(‘accuracy’) 
plt.xlabel(‘epoch’)
plt.legend()
plt.figure()
plt.plot(epochs, loss, ‘r’, label=’Training loss’)
plt.plot(epochs, val_loss, ‘b’, label=’Validation loss’)
plt.title(‘Training and validation loss’)
plt.ylabel(‘loss’) 
plt.xlabel(‘epoch’)
plt.legend()
plt.show()

In [ ]:
accuracy(cm)



In [ ]:
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements 